In [14]:
# Import needed libraries
import numpy as np # linear algebra 
import pandas as pd # data processing

import seaborn as sns 
import matplotlib as plt 
import sklearn 

In [15]:
# import Data
pccpdf=pd.read_csv("C:/Users/chunweil/Documents/pccp/Copy of wssc_0801.csv")
# Specify all the features(including target and predictors)
Important_features=['Diameter','TotalWB',"RepairReplacement",'AFO','FEA_WBYield',"Cover","Pressure","Manufacturer","WireClass","CylGg"]
# Create dataset for model by subsetting the original datset 
pccpdf1=pccpdf[Important_features]

# Check information of raw data
pccpdf1.describe()
pccpdf1.info()
pccpdf1.head(5)
pccpdf1.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22852 entries, 0 to 22851
Data columns (total 10 columns):
Diameter             22838 non-null object
TotalWB              22839 non-null float64
RepairReplacement    1188 non-null object
AFO                  22453 non-null float64
FEA_WBYield          20113 non-null float64
Cover                22431 non-null object
Pressure             22459 non-null float64
Manufacturer         12549 non-null object
WireClass            17844 non-null object
CylGg                18280 non-null float64
dtypes: float64(5), object(5)
memory usage: 1.3+ MB


Diameter                14
TotalWB                 13
RepairReplacement    21664
AFO                    399
FEA_WBYield           2739
Cover                  421
Pressure               393
Manufacturer         10303
WireClass             5008
CylGg                 4572
dtype: int64

In [16]:
# Data Preprocessing
# Fulter out the pipes with replaced and repairment record (we just want data with NULL value in RepairReplacement)
# Check the unique value of RepairReplacement column
pd.unique(pccpdf1.RepairReplacement)
pccpdf1["RepairReplacement"]=pccpdf1["RepairReplacement"].replace(np.nan,"NULL")
pccpdf1=pccpdf1.loc[pccpdf1["RepairReplacement"]=="NULL",:]
pccpdf1=pccpdf1.drop("RepairReplacement",axis=1)

C:\Users\chunweil\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [17]:
# Data cleaning and transformation 
# WireClass
pccpdf1["WireClass"]=pccpdf1["WireClass"].replace('UNKNOWN','Unknown')
pccpdf1["WireClass"]=pccpdf1["WireClass"].replace("III-1/2","I-III")
pccpdf1["WireClass"]=pccpdf1["WireClass"].replace("III","I-III")
pccpdf1["WireClass"]=pccpdf1["WireClass"].replace("I","I-III")
pd.unique(pccpdf1["WireClass"])

array([nan, 'I-III', 'II', 'IV'], dtype=object)

In [18]:
# Manufacturer
pccpdf1["Manufacturer"]=pccpdf1["Manufacturer"].replace('Interpace',"Interpace-Lock Joint Pipe Co.")
pccpdf1["Manufacturer"]=pccpdf1["Manufacturer"].replace('INTERPACE',"Interpace-Lock Joint Pipe Co.")
pccpdf1["Manufacturer"]=pccpdf1["Manufacturer"].replace('Lock Joint Pipe Co.',"Interpace-Lock Joint Pipe Co.")
pccpdf1["Manufacturer"]=pccpdf1["Manufacturer"].replace('Lock Joint Pipe Products',"Interpace-Lock Joint Pipe Co.")
pccpdf1["Manufacturer"]=pccpdf1["Manufacturer"].replace('PRICE BROTHERS',"Price Brothers")
pd.unique(pccpdf1["Manufacturer"])

array([nan, 'Interpace-Lock Joint Pipe Co.', 'Price Brothers',
       'International Pipe & Ceramics Corporation'], dtype=object)

In [19]:
# CylGg
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace("1/4 inch",0.25)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace("3/16 inch",0.1875)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace(10,0.102)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace(12,0.080)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace(14,0.064)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace(16,0.051)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace(17,0.045)
pccpdf1["CylGg"]=pccpdf1["CylGg"].replace(18,0.040)
pccpdf1["CylGg"]=pccpdf1["CylGg"].astype(np.float)

In [20]:
# Diameter
pccpdf1["Diameter"]=pccpdf1["Diameter"].replace("@",np.nan)
pccpdf1["Diameter"]=pccpdf1["Diameter"].astype(np.float)

In [21]:
# Cover 
pccpdf1["Cover"]=pccpdf1["Cover"].replace("As-Built Illegible",np.nan)
pccpdf1["Cover"]=pccpdf1["Cover"].astype(np.float)

In [22]:
# Convert all values to float type except for "Manufacturer" and "WireClass"
pccpdf1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21664 entries, 2 to 22850
Data columns (total 9 columns):
Diameter        21655 non-null float64
TotalWB         21661 non-null float64
AFO             21277 non-null float64
FEA_WBYield     19731 non-null float64
Cover           21272 non-null float64
Pressure        21299 non-null float64
Manufacturer    12362 non-null object
WireClass       17463 non-null object
CylGg           17898 non-null float64
dtypes: float64(7), object(2)
memory usage: 1.5+ MB


In [23]:
# Check missing data
pccpdf1.isnull().sum()

Diameter           9
TotalWB            3
AFO              387
FEA_WBYield     1933
Cover            392
Pressure         365
Manufacturer    9302
WireClass       4201
CylGg           3766
dtype: int64

In [46]:
# Filling NA value with the mean value 
pccpdf1=pccpdf1.fillna(pccpdf1.mean())
pccpdf1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21664 entries, 2 to 22850
Data columns (total 9 columns):
Diameter        21664 non-null float64
TotalWB         21664 non-null float64
AFO             21664 non-null float64
FEA_WBYield     21664 non-null float64
Cover           21664 non-null float64
Pressure        21664 non-null float64
Manufacturer    12362 non-null object
WireClass       17463 non-null object
CylGg           21664 non-null float64
dtypes: float64(7), object(2)
memory usage: 1.5+ MB


In [24]:
# log transformation (Selecting variables with high skewed distribution)
features=['Diameter','TotalWB','AFO','FEA_WBYield',"Cover","Pressure","CylGg"]
x=range(0,7)
for i in x:
    pccpdf1[features[i]]=np.log1p(pccpdf1[features[i]])

In [25]:
# Dummy vaiables (Convert categorical variable into dummy)
# One-Hot encoding on selected features 
pccpdf1=pd.get_dummies(pccpdf1,columns=['Manufacturer','WireClass'])

In [26]:
# Check correlation with new dataset
pccpdf1.head()
a=pccpdf1.corr()
pd.DataFrame(a)
correlation_break = pd.DataFrame(a["TotalWB"])
correlation_break.sort_values(by = 'TotalWB', ascending = False)

,TotalWB
TotalWB,1.000000
AFO,0.613057
Diameter,0.058423
FEA_WBYield,0.041394
WireClass_II,0.031383
Manufacturer_Interpace-Lock Joint Pipe Co.,0.030033
Manufacturer_International Pipe & Ceramics Corporation,0.030024
WireClass_IV,0.004640
WireClass_I-III,-0.019900
CylGg,-0.024810
